# Investigating Disney Revenue and Box Office Data
By: Neda Mostafavi

# Dataset Description

# Exploratory data analysis of the Disney datasets

In this final project, I will investigate Disneys datasets and I want to know if some Genres make more revenues.
The disney dataset is composed of 5 tables, `disney-voice-actors.csv`, `disney-director.csv`, `disney-characters.csv`, `disney_revenue_1991-2016.csv` and `disney_movies_total_gross.csv`. Each table is stored in a `.csv` file and contains information about movies' name, the main character and voice actor for that character, name of the director, release date, name of hero, genre of each movie and some financial information such as revenue by its components and MPAA rating, total gross and inflation adjusted gross.
I will use the `disney_revenue_1991-2016` and `disney_movies_total_gross` tables formally described below:

* **disney_movies_total_gross.csv**
    * This dataset includes Disney movie box office gross and inflation adjustments data from 1937 to 2016.
   * Size of this dataset is 44.27 KB, displaying 6 columns, and 579 rows.
* **disney_revenue_1991-2016.csv**
    * This file includes information on . Size of this table is 1.04 KB, and it has 26 rows and 7 columns.
    * This is a Disney financial data chart which contains annual gross revenues by sections (includes studio entertainment, parks and resorts, etc.) from 1991-2016. The data are collected from the Disney annual report.

I n this project I want to work with 'disney_revenue_1991-2016.csv' and 'disney_movies_total_gross.csv' to answer my question.


# Methods and Results

I am mostly interested in looking at financial tables, so I choosed to work on disney revenue and box office sales. for doing this, I should do some transformations on both tables, to clean them, imput missing values and also change their format to work on it.  
First, I need to import nedded libraries.

In [13]:
import pandas as pd
import numpy as np
import altair as alt

At first I want to know which components of total revenue has the bigest share of the revenue.
As we can see, some of the columns have missing values. So we should handle our missing values first.


# Disney Movies Total Gross

In [14]:
gross = pd.read_csv('data/disney_movies_total_gross.csv', 
                    parse_dates = ['release_date'])
gross.head()

,movie_title,release_date,genre,MPAA_rating,total_gross,inflation_adjusted_gross
0,Snow White and the Seven Dwarfs,1937-12-21,Musical,G,"$184,925,485","$5,228,953,251"
1,Pinocchio,1940-02-09,Adventure,G,"$84,300,000","$2,188,229,052"
2,Fantasia,1940-11-13,Musical,G,"$83,320,000","$2,187,090,808"
3,Song of the South,1946-11-12,Adventure,G,"$65,000,000","$1,078,510,579"
4,Cinderella,1950-02-15,Drama,G,"$85,000,000","$920,608,730"


In [15]:
gross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 579 entries, 0 to 578
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   movie_title               579 non-null    object        
 1   release_date              579 non-null    datetime64[ns]
 2   genre                     562 non-null    object        
 3   MPAA_rating               523 non-null    object        
 4   total_gross               579 non-null    object        
 5   inflation_adjusted_gross  579 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 27.3+ KB


As we can see, in two columns, genre and MPAA_rating we have missing data. I want to know the percentage of missing values.

In [16]:
gross.isnull().mean()*100

movie_title                 0.000000
release_date                0.000000
genre                       2.936097
MPAA_rating                 9.671848
total_gross                 0.000000
inflation_adjusted_gross    0.000000
dtype: float64

After working with gross table, I noticed some transformations should be applied on this dataset. So, I write it in the format of a function as follow.
I want to know distribution of movie released during 1937 and 2016. So I need variable Year to be assigned to my table.
We can see that about 3% of genre column and 10% of MPAA_rating are null values. So we need imputation here. Although in the result of isnull() it semms that total_gross and inflation_adjusted_gross don't need to be imputed, but when I looked at them deeply, there were some rows with 0 values, so these two columns need to be imputed, too. For total_gross and inflation_adjusted_gross I will use mean of them and for MAPP_rating and genre, I will use 'PG' and 'Comedy',respectively, which has the most amount in each of them.
I want to check to see the box office situation but befroe that, I should transform total_gross and inflation_adjusted_gross, because both are strings and have $ and ','.
I wrote a function to do all these cleaning and transforming together.

In [17]:
from cleaned_gross import cleaned_gross

In [18]:
gross1 = cleaned_gross(gross)
gross1

,release_date,genre,MPAA_rating,total_gross,inflation_adjusted_gross,Year
0,1937-12-21,Musical,G,184925485.0,5.228953e+09,1937
1,1940-02-09,Adventure,G,84300000.0,2.188229e+09,1940
2,1940-11-13,Musical,G,83320000.0,2.187091e+09,1940
3,1946-11-12,Adventure,G,65000000.0,1.078511e+09,1946
4,1950-02-15,Drama,G,85000000.0,9.206087e+08,1950
...,...,...,...,...,...,...
574,2016-09-02,Drama,PG-13,12545979.0,1.254598e+07,2016
575,2016-09-23,Drama,PG,8874389.0,8.874389e+06,2016
576,2016-11-04,Adventure,PG-13,232532923.0,2.325329e+08,2016
577,2016-11-23,Adventure,PG,246082029.0,2.460820e+08,2016


In [19]:
#distribution of movies thorough years
release_dist_viz = alt.Chart(gross1, width=500, height=300).mark_bar().encode(
    x= 'Year:O',
    y=alt.Y('count()', title = 'Number of Released Movies')
        ).properties(title = 'Distribution of Released Movies from 1937 to 2016')
release_dist_viz

alt.Chart(...)

Now I want to see which genres and which MPAA ratings make more sales. 

In [20]:
box_office_total_gr_viz = alt.Chart(gross1, width=500, height=300).mark_bar().encode(
    x= 'Year:O',
    y=alt.Y('sum(total_gross)', title = 'Amount of Total Sales')
        ).properties(title = 'Amount of Box Office Gross Sales')
box_office_total_gr_viz

alt.Chart(...)

It seems in 2016 the amount of box office sales is the most one, but we should use inflation adjusted or deflated amount to have a correct insight.

In [21]:
box_office_total_adj_viz = alt.Chart(gross1, width=500, height=300).mark_bar().encode(
    x= 'Year:O',
    y=alt.Y('sum(inflation_adjusted_gross)', title = 'Amount of Inflation Adjusted Sales')
        ).properties(title = 'Amount of Box Office Inflation Adjusted Sales')
box_office_total_adj_viz

alt.Chart(...)

WOW! The result is totaly different! This bar chart shows that in 1937, Disney had the most box office sales.
In the following phase, I want to know which genre and which MPAA ratings have the most box office sales.
We saw before that MPAA_rating and genre have missing values, so, I should handle missing values first.

Now I want to know which MPAA rating sales more at the box office.
So, let extract relevant table and also see the visualizations.

In [22]:
mpaa_rating_viz = alt.Chart(gross1, width=500, height=300).mark_bar().encode(
    x=alt.X('MPAA_rating', sort = 'y', title='MPAA Rating Categories'),
    y= alt.Y('sum(inflation_adjusted_gross)',  title = 'Inflation Adjusted Sales')
                ).properties(title = 'Box Office Inflation Adjusted Sales by MPAA_R')
mpaa_rating_viz

alt.Chart(...)

In [23]:
box_office_sales_by_rating= pd.DataFrame(gross1.groupby('MPAA_rating')['inflation_adjusted_gross'].sum().sort_values())
box_office_sales_by_rating

,inflation_adjusted_gross
MPAA_rating,
Not Rated,8.996202e+08
R,5.641192e+09
PG-13,1.492754e+10
PG,2.224670e+10
G,2.504845e+10


It is obvious that G rating has the most success in box office during this period of time.

In [24]:
genre_viz = alt.Chart(gross1, width=500, height=300).mark_bar().encode(
    x=alt.X('genre', sort= 'y', title='Movie Genre'),
    y=alt.Y('sum(inflation_adjusted_gross)', title='Inflation Adjusted Sales')
).properties(
    title='Box Office Inflation Adjusted Sales by Genre'
)

genre_viz

alt.Chart(...)

In [25]:
box_office_sales_by_genre= pd.DataFrame(gross1.groupby('genre')['inflation_adjusted_gross'].sum().sort_values())
box_office_sales_by_genre

,inflation_adjusted_gross
genre,
Concert/Performance,1.148217e+08
Horror,1.404831e+08
Black Comedy,1.567305e+08
Documentary,2.034884e+08
Western,5.167099e+08
Romantic Comedy,1.788873e+09
Thriller/Suspense,2.151691e+09
Action,5.498937e+09
Drama,8.195804e+09


As we can see, Adventure movies were the most successfull movies during this period, and after that,  Comedy movies were in the second position.


# Revenue 

In [26]:
revenue = pd.read_csv('data/disney_revenue_1991-2016.csv')
revenue.head()

,Year,Studio Entertainment[NI 1],Disney Consumer Products[NI 2],Disney Interactive[NI 3][Rev 1],Walt Disney Parks and Resorts,Disney Media Networks,Total
0,1991,2593.0,724.0,NaN,2794.0,NaN,6111
1,1992,3115.0,1081.0,NaN,3306.0,NaN,7502
2,1993,3673.4,1415.1,NaN,3440.7,NaN,8529
3,1994,4793.0,1798.2,NaN,3463.6,359,10414
4,1995,6001.5,2150.0,NaN,3959.8,414,12525


In [27]:
revenue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 7 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Year                             26 non-null     int64  
 1   Studio Entertainment[NI 1]       25 non-null     float64
 2   Disney Consumer Products[NI 2]   24 non-null     float64
 3   Disney Interactive[NI 3][Rev 1]  12 non-null     float64
 4   Walt Disney Parks and Resorts    26 non-null     float64
 5   Disney Media Networks            23 non-null     object 
 6   Total                            26 non-null     int64  
dtypes: float64(4), int64(2), object(1)
memory usage: 1.5+ KB


In [28]:
revenue.columns

Index(['Year', 'Studio Entertainment[NI 1]', 'Disney Consumer Products[NI 2]',
       'Disney Interactive[NI 3][Rev 1]', 'Walt Disney Parks and Resorts',
       'Disney Media Networks', 'Total'],
      dtype='object')

In [29]:
missing_percentage = revenue.isnull().mean() *100
misiing_frequency = revenue.isnull().sum()
print(missing_percentage)
print(misiing_frequency)

Year                                0.000000
Studio Entertainment[NI 1]          3.846154
Disney Consumer Products[NI 2]      7.692308
Disney Interactive[NI 3][Rev 1]    53.846154
Walt Disney Parks and Resorts       0.000000
Disney Media Networks              11.538462
Total                               0.000000
dtype: float64
Year                                0
Studio Entertainment[NI 1]          1
Disney Consumer Products[NI 2]      2
Disney Interactive[NI 3][Rev 1]    14
Walt Disney Parks and Resorts       0
Disney Media Networks               3
Total                               0
dtype: int64


We can see that 53% of information of 'Disney Interactive[NI 3][Rev 1]' are null. When I look deeper at the data, I saw that all missing values in this column can be calculated bu subtracting those columns with data, except for indexes 5 and 25. I will impute Studio Entertainment[NI 1], Disney Consumer Products[NI 2]  and Disney Media Networks with mean of each column and then calculate Disney Interactive[NI 3][Rev 1].


In [37]:
def cleaned_revenue(dirty_df):
    """
    Clean revenue data by selecting specific columns, renaming them, 
    converting 'Disney Media Networks' column to float, converting 'Year' column to string,
    filling NaN values with mean for each column separately, and calculating missing values for 'Disney Interactive' column.

    Parameters:
    dirty_df (DataFrame): The DataFrame containing revenue data.

    Returns:
    DataFrame: A cleaned DataFrame with selected columns, renamed columns, 
               'Disney Media Networks' column converted to float, 'Year' column converted to string,
               NaN values filled with mean for each column separately, and missing values calculated for 'Disney Interactive' column.
    """
    cleaned_df = dirty_df[['Year', 'Studio Entertainment[NI 1]', 'Disney Consumer Products[NI 2]',
                           'Disney Interactive[NI 3][Rev 1]', 'Walt Disney Parks and Resorts',
                           'Disney Media Networks', 'Total']]
    cleaned_df = cleaned_df.rename(columns={'Studio Entertainment[NI 1]': 'Studio_Entertainment',
                                            'Disney Consumer Products[NI 2]': 'Disney_Consumer_Products',
                                            'Disney Interactive[NI 3][Rev 1]': 'Disney_Interactive',
                                            'Walt Disney Parks and Resorts': 'Walt_Disney_Parks_and_Resorts',
                                            'Disney Media Networks': 'Disney_Media_Networks'})
    cleaned_df['Disney_Media_Networks'] = cleaned_df['Disney_Media_Networks'].str.replace(',', '').astype(float)
    cleaned_df['Year'] = cleaned_df['Year'].astype(str)

    # Fill missing values with mean for each column separately
    columns_to_fill = ['Studio_Entertainment', 'Disney_Consumer_Products', 'Walt_Disney_Parks_and_Resorts']
    for column in columns_to_fill:
        cleaned_df[column] = cleaned_df[column].fillna(cleaned_df[column].mean().round(1))
    
    # Fill Disney_Media_Networks with 0.
    cleaned_df['Disney_Media_Networks'] = cleaned_df['Disney_Media_Networks'].fillna(0)
    # Calculate missing values for 'Disney Interactive' column
    subtracted_total = cleaned_df['Total'] - cleaned_df[['Studio_Entertainment', 
                                                         'Disney_Consumer_Products', 
                                                         'Walt_Disney_Parks_and_Resorts', 
                                                         'Disney_Media_Networks']].fillna(0).sum(axis=1)
    # Check if the result is less than or equal to zero
    cleaned_df['Disney_Interactive'] = subtracted_total.where(subtracted_total > 0, 0)

    return cleaned_df

In [38]:
revenue1 = cleaned_revenue(revenue)
revenue1

,Year,Studio_Entertainment,Disney_Consumer_Products,Disney_Interactive,Walt_Disney_Parks_and_Resorts,Disney_Media_Networks,Total
0,1991,2593.0,724.0,0.0,2794.0,0.0,6111
1,1992,3115.0,1081.0,0.0,3306.0,0.0,7502
2,1993,3673.4,1415.1,0.0,3440.7,0.0,8529
3,1994,4793.0,1798.2,0.2,3463.6,359.0,10414
4,1995,6001.5,2150.0,0.0,3959.8,414.0,12525
5,1996,6445.0,2591.1,1058.9,4502.0,4142.0,18739
6,1997,6981.0,3782.0,174.0,5014.0,6522.0,22473
7,1998,6849.0,3193.0,260.0,5532.0,7142.0,22976
8,1999,6548.0,3030.0,206.0,6106.0,7512.0,23402
9,2000,5994.0,2602.0,388.0,6803.0,9615.0,25402


Now I have a clean data, I want to know which components of revenue has to highest share in total revenue.

In [39]:
revenue1.columns

Index(['Year', 'Studio_Entertainment', 'Disney_Consumer_Products',
       'Disney_Interactive', 'Walt_Disney_Parks_and_Resorts',
       'Disney_Media_Networks', 'Total'],
      dtype='object')

In [40]:
columns_to_share = ['Studio_Entertainment', 'Disney_Consumer_Products',
                    'Disney_Interactive', 'Walt_Disney_Parks_and_Resorts',
                    'Disney_Media_Networks']

# Create a new DataFrame to store the shares
revenue_share = pd.DataFrame()

# Add the 'Year' column to the revenue_share DataFrame
revenue_share['Year'] = revenue['Year']

# Calculate the share for each column
for column in columns_to_share:
    revenue_share[column + '_share'] = revenue1[column] / revenue1['Total'] *100
    
revenue_share

,Year,Studio_Entertainment_share,Disney_Consumer_Products_share,Disney_Interactive_share,Walt_Disney_Parks_and_Resorts_share,Disney_Media_Networks_share
0,1991,42.431681,11.847488,0.000000,45.720831,0.000000
1,1992,41.522261,14.409491,0.000000,44.068248,0.000000
2,1993,43.069527,16.591629,0.000000,40.341189,0.000000
3,1994,46.024582,17.267140,0.001920,33.259074,3.447283
4,1995,47.916168,17.165669,0.000000,31.615170,3.305389
5,1996,34.393511,13.827312,5.650782,24.024761,22.103634
6,1997,31.063943,16.829084,0.774262,22.311218,29.021492
7,1998,29.809366,13.897110,1.131616,24.077298,31.084610
8,1999,27.980514,12.947611,0.880267,26.091787,32.099821
9,2000,23.596567,10.243288,1.527439,26.781356,37.851350


In [41]:
revenue_share_melted = revenue_share.melt(id_vars='Year', var_name='Component', value_name='Share')

revenue_share_viz = alt.Chart(revenue_share_melted).mark_area().encode(
    x=alt.X('Year:O', title='Year'),
    y=alt.Y('sum(Share):Q', stack='normalize', title='Normalized Share'),
    color='Component:N',
    tooltip=['Year', 'Component', 'Share']
).properties(
    width=600,
    height=400,
    title='Stacked Area Chart of Revenue Components Over Years'
)

revenue_share_viz

alt.Chart(...)

As we can see, the structure of Disney revenue components has changed during 1991 to 2016. In 1991, there were just 3 components, and in 2016, it became 5 categories.
Disney media networks share has the highest share in recent years, but it introduced just in 1994 and at that time it had just 3 to 4 percents of revenue.
Walt Disney Parks and Resorts has the second highest share in 2016.

In [42]:
line_chart = alt.Chart(revenue_share_melted).mark_line().encode(
    x=alt.X('Year:O', title='Year'),
    y=alt.Y('Share:Q', title='Share'),
    color='Component:N',
    tooltip=['Year', 'Component', 'Share']
).properties(
    width=600,
    height=400,
    title='Revenue Components Over Years'
)

line_chart

alt.Chart(...)

This line chart gives us a good insight too. We can see that Studio Entertainment share declined during this period.

# Discussions

In this notebook, I analyzed two datasets releted to Walt Disney, one of them was information about its sales during 1937 to 2016 and the other one which was Disney's revenue by its components contained years 1991 to 2016. 
When I started working with these two datasets, I notices some missing values in both of them. Besides, I had to transform some columns in both tables. After doing transformation and imputation, I wanted to know which components of revenue has the highest share in the total revenue and if there is any changes in these shares during the time.
from the other dataset, I wanted to know which rating (MPAA rating) and genre is the most successfull one in terms of box office sale.

In the disney_movies_total_gross table, we have total_gross and inflation_adjusted_gross, which is the total gross information, deflated by the CPI. When I draw the bar chart with total_gross, the amount of box office sales increased from 1937 to 2016, but when I used inflation_adjusted_gross, it was clear that 1937 had the most box office sales.

For rating and genre, it figured out that rating 'G', and Adventurous genre has the most sales.


# References

* [Data Source](https://data.world/kgarrett/disney-character-success-00-16)
    * Thiese datasets are some information about Disney and is originaly belongs to ***Kelly Garrett***